In [1]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import json
import os
import cv2

def load_coco_annotations(json_path):
    """
    Load and parse COCO-style annotations from a JSON file.

    Parameters:
    - json_path: Path to the COCO annotations JSON file.

    Returns:
    - A dictionary mapping image filenames to their corresponding bounding boxes.
    """
    with open(json_path, 'r') as file:
        coco = json.load(file)

    annotations = {}
    for ann in coco['annotations']:
        image_id = ann['image_id']
        category_id = ann['category_id']
        
        # Assuming you're interested in a specific category (e.g., license plates)
        # If category_id matches license plates, adjust this ID according to your dataset
        if category_id == 1:  # Adjust this ID based on your COCO categories
            bbox = ann['bbox']  # COCO bbox format is [x_min, y_min, width, height]
            # Convert COCO bbox format to [x_min, y_min, x_max, y_max]
            bbox = [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]]
            
            # Get the corresponding image filename
            image_info = next((item for item in coco['images'] if item["id"] == image_id), None)
            if image_info:
                image_filename = image_info['file_name']
                if image_filename in annotations:
                    annotations[image_filename].append(bbox)
                else:
                    annotations[image_filename] = [bbox]

    return annotations


In [3]:
json_path = r'F:\workspace\705.603SP24\license_plate_recognition\annotations\coco_annotations.json'
image_dir = r'F:\workspace\705.603SP24\license_plate_recognition\license_plate_dataset'

annotations = load_coco_annotations(json_path)

# Prepare a list of image paths and their corresponding ground truths
test_images = []
ground_truths = []

for image_filename, bboxes in annotations.items():
    image_path = os.path.join(image_dir, image_filename)
    if os.path.exists(image_path):
        test_images.append(image_path)
        ground_truths.append(bboxes)


# LPR MODEL1: YOLOV3

In [4]:
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath('../'))

from model import Object_Detection_Model
from metrics import Metrics


lpr_model_path = (r"F:\workspace\705.603SP24\license_plate_recognition\yolo_lpr\lpr-yolov3.weights",
                  r"F:\workspace\705.603SP24\license_plate_recognition\yolo_lpr\lpr-yolov3.cfg")

# Initialize the model and metrics instances
model = Object_Detection_Model(lpr_model_path)
metrics = Metrics()

# Iterate over the dataset, predicting and updating metrics
for image_path, ground_truth in zip(test_images, ground_truths):
    image = cv2.imread(image_path)
    predictions = model.predict(image)
    metrics.update(predictions, ground_truth)

# Generate and display the final report
metrics.generate_report()


Metrics report generated: results\metrics_report.txt


# LPR MODEL2: YOLOV3-TINY

In [4]:
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath('../'))

from model import Object_Detection_Model
from metrics import Metrics
lpr_model_path2 = (r"F:\workspace\705.603SP24\license_plate_recognition\yolo_lpr\lpr-yolov3-tiny.weights",
                  r"F:\workspace\705.603SP24\license_plate_recognition\yolo_lpr\lpr-yolov3-tiny.cfg")

# Initialize the model and metrics instances
model2 = Object_Detection_Model(lpr_model_path2)
metrics = Metrics()

# Iterate over the dataset, predicting and updating metrics
for image_path, ground_truth in zip(test_images, ground_truths):
    image = cv2.imread(image_path)
    predictions = model2.predict(image)
    metrics.update(predictions, ground_truth)

# Generate and display the final report
metrics.generate_report()

Metrics report generated: results\metrics_report2.txt
